<a href="https://colab.research.google.com/github/sametkusdemir/SmartFarm_YieldPrediction/blob/main/notebooks/2_Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

# 1. Temiz veriyi yükle
file_path = '/content/drive/MyDrive/SmartFarm_Project/data/cleaned_data.csv'
df = pd.read_csv(file_path)

# 2. Kategorik ve Sayısal Değişkenleri Ayır
# Modelin 'Area' (Ülke) ismini ezberlemesini istemiyorsak silebiliriz,
# ancak ülke bazlı toprak farkı önemlidir. Şimdilik 'Continent' var diye Area'yı atabiliriz
# ya da model karmaşıklaşmasın diye sadece 'Continent' kullanabiliriz.
# Baseline için basit tutalım: Area'yı çıkarıp sadece Continent kullanalım.

df_baseline = df.drop(['Area', 'Year'], axis=1) # Yıl ve Ülke ismini çıkarıyoruz (Basitleştirme)

# 3. One-Hot Encoding (Kategorik verileri 0-1 matrisine çevirme)
# Item, Continent, Temp_Category sütunları sayısal olacak.
df_encoded = pd.get_dummies(df_baseline, columns=['Item', 'Continent', 'Temp_Category'], drop_first=True)

print("Encoding sonrası sütun sayısı:", df_encoded.shape[1])
display(df_encoded.head())

Encoding sonrası sütun sayısı: 21


,hg/ha_yield,average_rain_fall_mm_per_year,pesticides_tonnes,avg_temp,Rain_Temp_Ratio,Item_Maize,Item_Plantains and others,Item_Potatoes,"Item_Rice, paddy",Item_Sorghum,...,Item_Sweet potatoes,Item_Wheat,Item_Yams,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America,Temp_Category_Hot,Temp_Category_Mild
0,36613,1485.0,121.0,16.37,90.714722,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True
1,66667,1485.0,121.0,16.37,90.714722,False,False,True,False,False,...,False,False,False,False,True,False,False,False,False,True
2,23333,1485.0,121.0,16.37,90.714722,False,False,False,True,False,...,False,False,False,False,True,False,False,False,False,True
3,12500,1485.0,121.0,16.37,90.714722,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
4,7000,1485.0,121.0,16.37,90.714722,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,True


In [2]:
# X: Girdiler (Hedef hariç her şey)
X = df_encoded.drop('hg/ha_yield', axis=1)

# y: Hedef (Tahmin etmek istediğimiz verim)
y = df_encoded['hg/ha_yield']

# %80 Train, %20 Test ayrımı
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim Seti Boyutu: {X_train.shape}")
print(f"Test Seti Boyutu: {X_test.shape}")

Eğitim Seti Boyutu: (20745, 20)
Test Seti Boyutu: (5187, 20)


In [3]:
from sklearn.tree import DecisionTreeRegressor

# Modeli oluştur (random_state sonuçların hep aynı çıkmasını sağlar)
model = DecisionTreeRegressor(random_state=42)

# Modeli eğit
print("Model eğitiliyor...")
model.fit(X_train, y_train)
print(" Eğitim tamamlandı.")

Model eğitiliyor...
 Eğitim tamamlandı.


In [4]:
# Test seti üzerinde tahmin yap
y_pred = model.predict(X_test)

# Skorları hesapla
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("-" * 30)
print(f"Baseline Model Performansı:")
print(f"Ortalama Hata (MAE): {mae:.2f}")
print(f"R2 Skoru: {r2:.4f}")
print("-" * 30)

------------------------------
Baseline Model Performansı:
Ortalama Hata (MAE): 7206.33
R2 Skoru: 0.9495
------------------------------
